In [14]:
cd '/content/drive/MyDrive/Ali Amini/Fold1'

/content/drive/.shortcut-targets-by-id/1Vna0Aq4dpJCJlv4V9iEavYIq0Rokbbqv/Fold1


In [15]:
!pip install scikit-optimize
!pip install pyriemann

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
from tensorflow.keras import backend as K
K.set_image_data_format('channels_first')

In [17]:
def subject_classification1(y_true, y_pred, group, calculate_type = 'max_vote'):  
  """
  y_pred should be the output of 'model.predict' using 2 nodes in the dense layer and softmax activation.
  y_true should be in categorical mode. 
  """
  from pyriemann.utils.viz import plot_confusion_matrix
  from sklearn.metrics import fbeta_score

  #Categorical to normal labeling
  y_true = y_true.argmax(axis=-1)
  probability = np.array(y_pred)
  prediction = y_pred.argmax(axis = -1)
  max_vote = []
  subject_traget = []
  j = 0
  unique, counts = np.unique(group, return_counts=True)
  mean_ = np.zeros([len(unique),2], dtype='float32')
  for i in range(len(unique)):
    for k in range(2):
      mean_[i][k] = np.mean(probability[j:j+counts[i]-1,k])
    c = np.bincount(prediction[j:j+counts[i]-1])
    max_vote.append(np.argmax(c))
    subject_traget.append(y_true[j])
    j = j + counts[i]
  mean_            = mean_.argmax(axis = -1)
  max_vote         = np.array(max_vote)
  subject_traget   = np.array(subject_traget)
  f2_max_vote      = fbeta_score(subject_traget, max_vote, beta=0.5, average='binary')
  f2_mean          = fbeta_score(subject_traget, mean_, beta=0.5, average='binary')
  acc_max_vote     = np.mean(max_vote == subject_traget)
  acc_mean         = np.mean(mean_ == subject_traget)
  if calculate_type == 'max_vote':
    return acc_max_vote, f2_max_vote
  elif calculate_type == 'mean':
    return acc_mean, f2_mean
  else:
    raise ValueError('You have NOT entered an accurate type!')

In [18]:
def subject_classification(y_true, y_pred, group, calculate_type = 'max_vote'):  
  """
  y_pred should be the output of 'model.predict' using 2 nodes in the dense layer and softmax activation.
  y_true should be in categorical mode. 
  """
  from pyriemann.utils.viz import plot_confusion_matrix
  from sklearn.metrics import fbeta_score

  #Categorical to normal labeling
  y_true = y_true.argmax(axis=-1)
  probability = np.array(y_pred)
  prediction = y_pred.argmax(axis = -1)
  max_vote = []
  subject_traget = []
  j = 0
  unique, counts = np.unique(group, return_counts=True)
  mean_ = np.zeros([len(unique),2], dtype='float32')
  for i in range(len(unique)):
    for k in range(2):
      mean_[i][k] = np.mean(probability[j:j+counts[i]-1,k])
    c = np.bincount(prediction[j:j+counts[i]-1])
    max_vote.append(np.argmax(c))
    subject_traget.append(y_true[j])
    j = j + counts[i]
  mean_            = mean_.argmax(axis = -1)
  max_vote         = np.array(max_vote)
  subject_traget   = np.array(subject_traget)
  f2_max_vote      = fbeta_score(subject_traget, max_vote, beta=0.5, average='binary')
  f2_mean          = fbeta_score(subject_traget, mean_, beta=0.5, average='binary')
  acc_max_vote     = np.mean(max_vote == subject_traget)
  acc_mean         = np.mean(mean_ == subject_traget)
  if calculate_type == 'max_vote':
    names          = ['ADHD','Control']
    plt.figure(0)
    plot_confusion_matrix(subject_traget, max_vote, names, title = 'Max Vote Type')
    return acc_max_vote, f2_max_vote
  elif calculate_type == 'mean':
    names        = ['ADHD','Control']
    plt.figure(0)
    plot_confusion_matrix(subject_traget, mean_, names, title = 'Mean Type')
    return acc_mean, f2_mean
  else:
    raise ValueError('You have NOT entered an accurate type!')

In [19]:
import numpy as np

def augment_time_series_data(data, label, sigma, magnification_factor):
    augmented_data = []
    augmented_label = []

    n, _, electrodes, time_steps = data.shape
    if sigma[0]==0 and magnification_factor[0]==0:
      return data,label
    else:
      # loop through each sample
      for i in range(n):
          for s in sigma:
              for m in magnification_factor:
                  # generate noise with the given sigma
                  noise = np.random.normal(0, s, (electrodes, time_steps))
                  
                  # augment the data with the noise and magnification factor
                  augmented_sample = data[i, :, :, :] + noise * m
                  augmented_data.append(augmented_sample)
                  
                  # augment the label with the same label
                  augmented_label.append(label[i])
      # change list to numpy array
      augmented_data, augmented_label = np.array(augmented_data), np.array(augmented_label)

      return np.concatenate((data,augmented_data)), np.concatenate((label,augmented_label))

In [20]:
from itertools import combinations
def generate_combinations(magnification_factor, sigma):
    result = [{"magnification_factor": [0], "sigma": [0]}]
    for i in range(1, 4):
        for m_comb in combinations(magnification_factor, i):
            for sigma_comb in combinations(sigma, i):
                result.append({"magnification_factor": list(m_comb), "sigma": list(sigma_comb)})
    return result

In [21]:
def sample_classification(y_true, y_pred):
  """
  y_pred should be the output of 'model.predict' using 2 nodes in the dense layer and softmax activation.
  y_true should be in categorical mode. 
  """
  preds       = y_pred.argmax(axis = -1)  
  acc         = np.mean(preds == y_true.argmax(axis=-1))
  f2 = fbeta_score(y_true.argmax(axis=-1), preds, beta=0.5, average='binary') 
  return acc, f2


In [22]:
def ClassWeightSoftmax(y):
  class_weights = compute_class_weight(class_weight = "balanced",classes = np.unique(Y_train), y = Y_train)
  class_weights = dict(zip(np.unique(Y_train), class_weights))
  class_weights[0] = int(10*round(class_weights[0],1))
  class_weights[1] = int(10*round(class_weights[1],1))
  class_weight = [{0: 1, 1: class_weights[0]}, {0: 1, 1: class_weights[1]}]
  print("The Class Weight is:",class_weight)
  return class_weight

In [23]:
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import utils as np_utils
from sklearn.model_selection import StratifiedGroupKFold, GridSearchCV
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout, multiply,LSTM
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.optimizers.experimental import AdamW
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import GlobalAveragePooling2D
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import fbeta_score
import math
import gc
from numpy import mean
from numpy import std
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import cross_val_score
from skopt.utils import use_named_args
from skopt import gp_minimize
import tensorflow as tf
import numpy as np

def TactileNet(nb_classes=2, Chans = 19, Samples = 512, kernLength = 16, F2 = 64,
               F1 = 64, D=4, dropoutRate = 0.5,dropoutType = 'Dropout', norm_rate = 0.25,
               Dense_nodes = 16, optimizer_type = 'Adam', lr=0.001, **kwargs):


  if dropoutType == 'SpatialDropout2D':
      dropoutType = SpatialDropout2D
  elif dropoutType == 'Dropout':
      dropoutType = Dropout
  else:
      raise ValueError('dropoutType must be one of SpatialDropout2D '
                        'or Dropout, passed as a string.')
  #EEGNet alike part
  input1       = Input(shape = (1, Chans, Samples))
  block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                  input_shape = (1, Chans, Samples),
                                  use_bias = False)(input1)
  block1       = BatchNormalization(axis = 1 ,trainable = True)(block1)
  block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                  depth_multiplier = D,
                                  depthwise_constraint = max_norm(norm_rate))(block1)
  block1       = BatchNormalization( axis = 1, trainable = True)(block1)
  block1       = Activation('elu')(block1)
  block1       = AveragePooling2D((1, 2))(block1)
  block1       = dropoutType(dropoutRate)(block1)

  ###############################################
  #first tower
  sub_block1   = Conv2D(64, (1, 1), padding = 'same',use_bias = False)(block1)
  sub_block1   = SeparableConv2D(128, (1, 128), padding = 'same',use_bias = False)(sub_block1)
  sub_block1   = AveragePooling2D((1, 2), padding = 'same')(sub_block1)
  #second tower
  sub_block2   = Conv2D(16, (1, 1), padding = 'same',use_bias = False)(block1)
  sub_block2   = SeparableConv2D(32, (1, 256), padding = 'same',use_bias = False)(sub_block2)
  sub_block2  = AveragePooling2D((1, 2), padding = 'same')(sub_block2)
  #third tower
  sub_block3   = Conv2D(64, (1, 1), padding = 'same', strides=(1,2), use_bias = False)(block1)
  #forth tower
  sub_block4   = AveragePooling2D((1, 2), padding = 'same')(block1)
  sub_block4   = Conv2D(32, (1, 1), padding = 'same',use_bias = False)(sub_block4)
  #concatenation
  concat       = concatenate([sub_block1, sub_block2, sub_block4, sub_block3],axis=1)
  
  #last tower
  block2       = BatchNormalization( axis = 1, trainable = True)(concat)
  block2       = Activation('elu')(block2)
  #SENEt block
  squeeze1     = GlobalAveragePooling2D()(block2)
  excitation1  = Dense(Dense_nodes, activation='relu')(squeeze1)
  excitation1  = Dense(256, activation='sigmoid')(excitation1)
  block2       = Permute(dims=(2,3,1))(block2)
  excitation1  = multiply([block2, excitation1])
  excitation1  = Permute(dims=(3,1,2))(excitation1)

  block2       = SeparableConv2D(256, (1, 64), padding = 'same',use_bias = False)(excitation1)
  block2       = BatchNormalization( axis = 1, trainable = True)(block2)
  block2       = Activation('elu')(block2)
  #SENEt block
  squeeze2     = GlobalAveragePooling2D()(block2)
  excitation2  = Dense(Dense_nodes, activation='relu')(squeeze2)
  excitation2  = Dense(256, activation='sigmoid')(excitation2)
  block2       = Permute(dims=(2,3,1))(block2)
  excitation2  = multiply([block2, excitation2])
  excitation2  = Permute(dims=(3,1,2))(excitation2)

  block2       = dropoutType(dropoutRate)(excitation2)

  GB           = GlobalAveragePooling2D()(block2)
  dense        = Dense(nb_classes, name = 'dense',kernel_constraint = max_norm(norm_rate))(GB)
  softmax      = Activation('softmax', name = 'softmax')(dense)
  if optimizer_type == 'Adam':
    optimizer = Adam(learning_rate = lr)
  if optimizer_type == 'Adamax':
    optimizer = Adamax(learning_rate = lr)
  if optimizer_type == 'AdamW':
    optimizer = AdamW(learning_rate = lr)
  model       = Model(inputs=input1, outputs=softmax)
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics = ['accuracy'])
  return model

In [24]:
import numpy as np
X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')

X_test = np.load('X_test.npy')
Y_test = np.load('Y_test.npy')

Y_test  = np.expand_dims(Y_test,axis=1)
Y_test  = np_utils.to_categorical(Y_test)

Y_train  = np.expand_dims(Y_train,axis=1)
Y_train  = np_utils.to_categorical(Y_train)

Group_train = np.load('Group_train.npy')
Group_test = np.load('Group_test.npy')

print("X_train shape:",X_train.shape)
print("Y_train shape:",Y_train.shape)

print("X_test shape:",X_train.shape)
print("Y_test shape:",Y_test.shape)

print("Group_train shape:",Group_train.shape)
print("Group_test shape:",Group_test.shape)

X_train shape: (3695, 1, 19, 512)
Y_train shape: (3695, 2)
X_test shape: (3695, 1, 19, 512)
Y_test shape: (478, 2)
Group_train shape: (3695, 1)
Group_test shape: (478, 1)


In [25]:
# define the space of hyperparameters to search
search_space = list()
search_space.append(Categorical([8, 16, 32, 64], name='F1'))
search_space.append(Categorical([16, 32, 64, 128, 256], name='F2'))
search_space.append(Categorical([2, 4, 8], name='D'))
search_space.append(Categorical([32, 64, 128, 256, 512], name='kernLength'))
search_space.append(Categorical([8,16,32], name='Dense_nodes'))
search_space.append(Categorical([0.5, 1.0, 5.0, 10.0], name='norm_rate'))
search_space.append(Categorical([0.0001, 0.0005, 0.001, 0.005, 0.01], name='lr'))
search_space.append(Categorical(['Adam','Adamax','AdamW'], name='optimizer'))
search_space.append(Categorical([0.3, 0.4, 0.5], name='dropoutRate'))
search_space.append(Categorical([32, 64, 128], name='batch_size'))
# define the function used to evaluate a given configuration
@use_named_args(search_space)
def evaluate_model(**params):
  sample_acc = []
  sample_f2  = []
  subject_acc = []
  subject_f2 = []
  sample_loss = []
  ### Defining SKGF ###
  #######################
  # Callbacks #
  path = '/content/drive/MyDrive/Ali Amini/Fold1/TactileNet/Bayesian Saved Model/'
  checkpoint = ModelCheckpoint(path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
  keras_reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00005, verbose=2)  
  callbacks = [checkpoint,es,keras_reduce_lr]
  model     = TactileNet(**params)
  # Fitting ...
  fit       = model.fit(X_train, Y_train,
                          verbose = 2,shuffle=True,
                          epochs=100,batch_size=params['batch_size'],
                          validation_data =(X_test, Y_test),
                          callbacks=callbacks)
  del model, fit
  ### Loading The Best Saved Model ###
  saved_model = load_model(path)
  # Predict The model
  test_predict = saved_model.predict(X_test)
  test_loss, test_acc = saved_model.evaluate(X_test, Y_test)
  ######################################################################################################
  print("test_loss",test_loss)
  print('test_acc',test_acc)
  ######################################################################################################
  sub_acc,sub_f2 = subject_classification1(Y_test, test_predict, Group_test, calculate_type = 'max_vote')
  sam_acc,sam_f2 = sample_classification(Y_test, test_predict)
  # convert from a maximizing score to a minimizing score
  return 1.0 - sam_acc

In [26]:
n_iteration = 100
# perform optimization
result = gp_minimize(evaluate_model, search_space, n_calls=n_iteration)
# summarizing finding:
print('Best Accuracy: %.3f' % (1.0 - result.fun))
print('Best Parameters: %s' % (result.x))

Epoch 1/100

Epoch 1: val_accuracy improved from -inf to 0.24477, saving model to /content/drive/MyDrive/Ali Amini/Fold1/TactileNet/Bayesian Saved Model/


116/116 - 30s - loss: 0.3057 - accuracy: 0.8601 - val_loss: 1.6005 - val_accuracy: 0.2448 - lr: 0.0100 - 30s/epoch - 258ms/step
Epoch 2/100

Epoch 2: val_accuracy improved from 0.24477 to 0.94770, saving model to /content/drive/MyDrive/Ali Amini/Fold1/TactileNet/Bayesian Saved Model/


116/116 - 6s - loss: 0.1463 - accuracy: 0.9437 - val_loss: 0.1039 - val_accuracy: 0.9477 - lr: 0.0100 - 6s/epoch - 53ms/step
Epoch 3/100

Epoch 3: val_accuracy did not improve from 0.94770
116/116 - 2s - loss: 0.1108 - accuracy: 0.9572 - val_loss: 0.7688 - val_accuracy: 0.8159 - lr: 0.0100 - 2s/epoch - 17ms/step
Epoch 4/100

Epoch 4: val_accuracy did not improve from 0.94770
116/116 - 2s - loss: 0.1208 - accuracy: 0.9556 - val_loss: 1.5883 - val_accuracy: 0.7259 - lr: 0.0100 - 2s/epoch - 14ms/step
Epoch 5/100

Epoch 5: val_accuracy did not improve from 0.94770
116/116 - 2s - loss: 0.0819 - accuracy: 0.9662 - val_loss: 3.0124 - val_accuracy: 0.5586 - lr: 0.0100 - 2s/epoch - 14ms/step
Epoch 6/100

Epoch 6: val_accuracy did not improve from 0.94770
116/116 - 2s - loss: 0.0815 - accuracy: 0.9691 - val_loss: 0.5562 - val_accuracy: 0.7971 - lr: 0.0100 - 2s/epoch - 14ms/step
Epoch 7/100

Epoch 7: val_accuracy did not improve from 0.94770
116/116 - 2s - loss: 0.0803 - accuracy: 0.9708 - val_lo

116/116 - 14s - loss: 0.2627 - accuracy: 0.8847 - val_loss: 0.7522 - val_accuracy: 0.3975 - lr: 0.0010 - 14s/epoch - 123ms/step
Epoch 2/100

Epoch 2: val_accuracy improved from 0.39749 to 0.80335, saving model to /content/drive/MyDrive/Ali Amini/Fold1/TactileNet/Bayesian Saved Model/


116/116 - 6s - loss: 0.0917 - accuracy: 0.9627 - val_loss: 0.4911 - val_accuracy: 0.8033 - lr: 0.0010 - 6s/epoch - 48ms/step
Epoch 3/100

Epoch 3: val_accuracy did not improve from 0.80335
116/116 - 3s - loss: 0.0498 - accuracy: 0.9832 - val_loss: 1.0168 - val_accuracy: 0.7887 - lr: 0.0010 - 3s/epoch - 22ms/step
Epoch 4/100

Epoch 4: val_accuracy did not improve from 0.80335
116/116 - 3s - loss: 0.0359 - accuracy: 0.9876 - val_loss: 0.5670 - val_accuracy: 0.7950 - lr: 0.0010 - 3s/epoch - 26ms/step
Epoch 5/100

Epoch 5: val_accuracy did not improve from 0.80335
116/116 - 2s - loss: 0.0578 - accuracy: 0.9819 - val_loss: 0.7904 - val_accuracy: 0.7950 - lr: 0.0010 - 2s/epoch - 18ms/step
Epoch 6/100

Epoch 6: val_accuracy improved from 0.80335 to 0.88912, saving model to /content/drive/MyDrive/Ali Amini/Fold1/TactileNet/Bayesian Saved Model/


...

29/29 - 12s - loss: 0.6417 - accuracy: 0.6541 - val_loss: 0.6880 - val_accuracy: 0.6402 - lr: 1.0000e-04 - 12s/epoch - 425ms/step
Epoch 2/100

Epoch 2: val_accuracy did not improve from 0.64017
29/29 - 3s - loss: 0.5097 - accuracy: 0.7632 - val_loss: 0.6822 - val_accuracy: 0.6297 - lr: 1.0000e-04 - 3s/epoch - 106ms/step
Epoch 3/100

Epoch 3: val_accuracy improved from 0.64017 to 0.98326, saving model to /content/drive/MyDrive/Ali Amini/Fold1/TactileNet/Bayesian Saved Model/


29/29 - 8s - loss: 0.3350 - accuracy: 0.8566 - val_loss: 0.6786 - val_accuracy: 0.9833 - lr: 1.0000e-04 - 8s/epoch - 267ms/step
Epoch 4/100

Epoch 4: val_accuracy did not improve from 0.98326
29/29 - 3s - loss: 0.2269 - accuracy: 0.9150 - val_loss: 0.6728 - val_accuracy: 0.4686 - lr: 1.0000e-04 - 3s/epoch - 107ms/step
Epoch 5/100

Epoch 5: val_accuracy did not improve from 0.98326
29/29 - 3s - loss: 0.1718 - accuracy: 0.9461 - val_loss: 0.6561 - val_accuracy: 0.9100 - lr: 1.0000e-04 - 3s/epoch - 105ms/step
Epoch 6/100

Epoch 6: val_accuracy did not improve from 0.98326
29/29 - 3s - loss: 0.1264 - accuracy: 0.9691 - val_loss: 0.6330 - val_accuracy: 0.5795 - lr: 1.0000e-04 - 3s/epoch - 105ms/step
Epoch 7/100

Epoch 7: val_accuracy improved from 0.98326 to 0.99372, saving model to /content/drive/MyDrive/Ali Amini/Fold1/TactileNet/Bayesian Saved Model/


29/29 - 7s - loss: 0.0927 - accuracy: 0.9762 - val_loss: 0.5699 - val_accuracy: 0.9937 - lr: 1.0000e-04 - 7s/epoch - 228ms/step
Epoch 8/100

Epoch 8: val_accuracy did not improve from 0.99372
29/29 - 3s - loss: 0.0728 - accuracy: 0.9827 - val_loss: 0.4589 - val_accuracy: 0.9038 - lr: 1.0000e-04 - 3s/epoch - 109ms/step
Epoch 9/100

Epoch 9: val_accuracy did not improve from 0.99372
29/29 - 3s - loss: 0.0577 - accuracy: 0.9884 - val_loss: 0.4471 - val_accuracy: 0.9707 - lr: 1.0000e-04 - 3s/epoch - 106ms/step
Epoch 10/100

Epoch 10: val_accuracy improved from 0.99372 to 0.99582, saving model to /content/drive/MyDrive/Ali Amini/Fold1/TactileNet/Bayesian Saved Model/


29/29 - 7s - loss: 0.0502 - accuracy: 0.9873 - val_loss: 0.3001 - val_accuracy: 0.9958 - lr: 1.0000e-04 - 7s/epoch - 248ms/step
Epoch 11/100

Epoch 11: val_accuracy did not improve from 0.99582
29/29 - 3s - loss: 0.0388 - accuracy: 0.9913 - val_loss: 0.2533 - val_accuracy: 0.9958 - lr: 1.0000e-04 - 3s/epoch - 108ms/step
Epoch 12/100

Epoch 12: val_accuracy improved from 0.99582 to 0.99791, saving model to /content/drive/MyDrive/Ali Amini/Fold1/TactileNet/Bayesian Saved Model/


29/29 - 8s - loss: 0.0318 - accuracy: 0.9946 - val_loss: 0.1188 - val_accuracy: 0.9979 - lr: 1.0000e-04 - 8s/epoch - 279ms/step
Epoch 13/100

Epoch 13: val_accuracy did not improve from 0.99791
29/29 - 3s - loss: 0.0276 - accuracy: 0.9951 - val_loss: 0.2155 - val_accuracy: 0.9603 - lr: 1.0000e-04 - 3s/epoch - 107ms/step
Epoch 14/100

Epoch 14: val_accuracy did not improve from 0.99791
29/29 - 3s - loss: 0.0237 - accuracy: 0.9954 - val_loss: 0.0691 - val_accuracy: 0.9895 - lr: 1.0000e-04 - 3s/epoch - 106ms/step
Epoch 15/100

Epoch 15: val_accuracy did not improve from 0.99791
29/29 - 3s - loss: 0.0215 - accuracy: 0.9962 - val_loss: 0.0389 - val_accuracy: 0.9979 - lr: 1.0000e-04 - 3s/epoch - 105ms/step
Epoch 16/100

Epoch 16: val_accuracy did not improve from 0.99791
29/29 - 3s - loss: 0.0221 - accuracy: 0.9951 - val_loss: 0.0232 - val_accuracy: 0.9979 - lr: 1.0000e-04 - 3s/epoch - 107ms/step
Epoch 17/100

Epoch 17: val_accuracy did not improve from 0.99791
29/29 - 3s - loss: 0.0173 - ac

In [27]:
# Getting the best params
best = {}
for j in range(len(result.x)):
  best[search_space[j].name] = result.x[j]
#Preparing the data augmentation combination
magnification_factor = [1,2,3]
sigma = [0.1,0.01,0.001]
combinations = generate_combinations(magnification_factor, sigma)
aug_check = '/content/drive/MyDrive/Ali Amini/Fold1/TactileNet/Aug Saved Model/'
# Removing the last one
combinations.remove({'magnification_factor': [1, 2, 3], 'sigma': [0.1, 0.01, 0.001]})
#####################################################################################
for combination in combinations:
    print(combination)
# Training the data on each combination of augmentation and evaluate the moel on the test set
Result =[]
for i in range(len(combinations)):
  param = combinations[i]
  checkpoint_test = ModelCheckpoint(aug_check+'Config'+str(i+1)+'.check', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
  es_test = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
  keras_reduce_lr_test = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00005, verbose=2)  
  callbacks = [checkpoint_test,es_test,keras_reduce_lr_test]
  model = TactileNet(**best)
  augmented_data, augmented_label = augment_time_series_data(X_train,Y_train,**param)
  fit  = model.fit(augmented_data, augmented_label,verbose = 2,shuffle=True, 
                  epochs=100,batch_size=best['batch_size'],
                  validation_data =(X_test, Y_test),
                  callbacks=callbacks)
  del model,fit
  saved_model = load_model(aug_check+'Config'+str(i+1)+'.check')
  # Calculating Sample-Subject accuracy and F-measure
  test_predict_final = saved_model.predict(X_test)
  test_sub_acc_final,test_sub_f2_final = subject_classification1(Y_test, test_predict_final, Group_test, calculate_type = 'max_vote')
  test_sam_acc_final,test_sam_f2_final = sample_classification(Y_test, test_predict_final)
  # appending the results to Result
  temp ={'Sample Accuracy':test_sam_acc_final, 'Subject Accuracy':test_sub_acc_final, 'Sample F-measure': test_sam_f2_final, 'Subject F-measure': test_sub_f2_final}
  Result.append(temp)
  del saved_model, augmented_data, augmented_label
print("Finished!")
print(Result)

{'magnification_factor': [0], 'sigma': [0]}
{'magnification_factor': [1], 'sigma': [0.1]}
{'magnification_factor': [1], 'sigma': [0.01]}
{'magnification_factor': [1], 'sigma': [0.001]}
{'magnification_factor': [2], 'sigma': [0.1]}
{'magnification_factor': [2], 'sigma': [0.01]}
{'magnification_factor': [2], 'sigma': [0.001]}
{'magnification_factor': [3], 'sigma': [0.1]}
{'magnification_factor': [3], 'sigma': [0.01]}
{'magnification_factor': [3], 'sigma': [0.001]}
{'magnification_factor': [1, 2], 'sigma': [0.1, 0.01]}
{'magnification_factor': [1, 2], 'sigma': [0.1, 0.001]}
{'magnification_factor': [1, 2], 'sigma': [0.01, 0.001]}
{'magnification_factor': [1, 3], 'sigma': [0.1, 0.01]}
{'magnification_factor': [1, 3], 'sigma': [0.1, 0.001]}
{'magnification_factor': [1, 3], 'sigma': [0.01, 0.001]}
{'magnification_factor': [2, 3], 'sigma': [0.1, 0.01]}
{'magnification_factor': [2, 3], 'sigma': [0.1, 0.001]}
{'magnification_factor': [2, 3], 'sigma': [0.01, 0.001]}
Epoch 1/100

Epoch 1: val_acc

29/29 - 20s - loss: 0.4884 - accuracy: 0.7605 - val_loss: 0.6659 - val_accuracy: 0.8096 - lr: 5.0000e-04 - 20s/epoch - 695ms/step
Epoch 2/100

Epoch 2: val_accuracy did not improve from 0.80962
29/29 - 2s - loss: 0.2322 - accuracy: 0.9034 - val_loss: 0.6563 - val_accuracy: 0.4854 - lr: 5.0000e-04 - 2s/epoch - 72ms/step
Epoch 3/100

Epoch 3: val_accuracy did not improve from 0.80962
29/29 - 2s - loss: 0.1172 - accuracy: 0.9583 - val_loss: 0.6249 - val_accuracy: 0.4707 - lr: 5.0000e-04 - 2s/epoch - 66ms/step
Epoch 4/100

Epoch 4: val_accuracy improved from 0.80962 to 0.99163, saving model to /content/drive/MyDrive/Ali Amini/Fold1/TactileNet/Aug Saved Model/Config1.check


29/29 - 7s - loss: 0.0739 - accuracy: 0.9740 - val_loss: 0.5511 - val_accuracy: 0.9916 - lr: 5.0000e-04 - 7s/epoch - 229ms/step
Epoch 5/100

Epoch 5: val_accuracy did not improve from 0.99163
29/29 - 2s - loss: 0.0475 - accuracy: 0.9846 - val_loss: 0.4796 - val_accuracy: 0.9833 - lr: 5.0000e-04 - 2s/epoch - 70ms/step
Epoch 6/100

Epoch 6: val_accuracy did not improve from 0.99163
29/29 - 2s - loss: 0.0471 - accuracy: 0.9819 - val_loss: 0.6223 - val_accuracy: 0.4289 - lr: 5.0000e-04 - 2s/epoch - 67ms/step
Epoch 7/100

Epoch 7: val_accuracy did not improve from 0.99163
29/29 - 2s - loss: 0.0334 - accuracy: 0.9870 - val_loss: 0.6692 - val_accuracy: 0.4519 - lr: 5.0000e-04 - 2s/epoch - 67ms/step
Epoch 8/100

Epoch 8: val_accuracy did not improve from 0.99163
29/29 - 2s - loss: 0.0271 - accuracy: 0.9911 - val_loss: 0.2885 - val_accuracy: 0.8285 - lr: 5.0000e-04 - 2s/epoch - 67ms/step
Epoch 9/100

Epoch 9: val_accuracy did not improve from 0.99163
29/29 - 2s - loss: 0.0337 - accuracy: 0.9881

58/58 - 16s - loss: 0.3725 - accuracy: 0.8252 - val_loss: 0.6339 - val_accuracy: 0.9833 - lr: 5.0000e-04 - 16s/epoch - 281ms/step
Epoch 2/100

Epoch 2: val_accuracy did not improve from 0.98326
58/58 - 4s - loss: 0.1116 - accuracy: 0.9583 - val_loss: 0.5051 - val_accuracy: 0.9749 - lr: 5.0000e-04 - 4s/epoch - 67ms/step
Epoch 3/100

Epoch 3: val_accuracy did not improve from 0.98326
58/58 - 4s - loss: 0.0575 - accuracy: 0.9796 - val_loss: 0.6810 - val_accuracy: 0.3996 - lr: 5.0000e-04 - 4s/epoch - 65ms/step
Epoch 4/100

Epoch 4: val_accuracy did not improve from 0.98326
58/58 - 4s - loss: 0.0286 - accuracy: 0.9915 - val_loss: 0.3303 - val_accuracy: 0.9184 - lr: 5.0000e-04 - 4s/epoch - 65ms/step
Epoch 5/100

Epoch 5: val_accuracy did not improve from 0.98326
58/58 - 4s - loss: 0.0211 - accuracy: 0.9928 - val_loss: 0.0754 - val_accuracy: 0.9770 - lr: 5.0000e-04 - 4s/epoch - 67ms/step
Epoch 6/100

Epoch 6: val_accuracy improved from 0.98326 to 0.98954, saving model to /content/drive/MyDriv

58/58 - 7s - loss: 0.0191 - accuracy: 0.9931 - val_loss: 0.0414 - val_accuracy: 0.9895 - lr: 5.0000e-04 - 7s/epoch - 127ms/step
Epoch 7/100

Epoch 7: val_accuracy did not improve from 0.98954
58/58 - 4s - loss: 0.0124 - accuracy: 0.9957 - val_loss: 0.0527 - val_accuracy: 0.9833 - lr: 5.0000e-04 - 4s/epoch - 68ms/step
Epoch 8/100

Epoch 8: val_accuracy did not improve from 0.98954
58/58 - 4s - loss: 0.0096 - accuracy: 0.9974 - val_loss: 0.0504 - val_accuracy: 0.9728 - lr: 5.0000e-04 - 4s/epoch - 67ms/step
Epoch 9/100

Epoch 9: val_accuracy improved from 0.98954 to 0.99582, saving model to /content/drive/MyDrive/Ali Amini/Fold1/TactileNet/Aug Saved Model/Config2.check


...

145/145 - 13s - loss: 0.0092 - accuracy: 0.9966 - val_loss: 0.1003 - val_accuracy: 0.9456 - lr: 5.0000e-04 - 13s/epoch - 92ms/step
Epoch 5/100

Epoch 5: val_accuracy did not improve from 0.94561
145/145 - 9s - loss: 0.0054 - accuracy: 0.9982 - val_loss: 0.4370 - val_accuracy: 0.8619 - lr: 5.0000e-04 - 9s/epoch - 65ms/step
Epoch 6/100

Epoch 6: val_accuracy improved from 0.94561 to 0.95188, saving model to /content/drive/MyDrive/Ali Amini/Fold1/TactileNet/Aug Saved Model/Config19.check


145/145 - 13s - loss: 0.0048 - accuracy: 0.9986 - val_loss: 0.1367 - val_accuracy: 0.9519 - lr: 5.0000e-04 - 13s/epoch - 87ms/step
Epoch 7/100

Epoch 7: val_accuracy improved from 0.95188 to 0.96862, saving model to /content/drive/MyDrive/Ali Amini/Fold1/TactileNet/Aug Saved Model/Config19.check


145/145 - 13s - loss: 0.0074 - accuracy: 0.9975 - val_loss: 0.1364 - val_accuracy: 0.9686 - lr: 5.0000e-04 - 13s/epoch - 90ms/step
Epoch 8/100

Epoch 8: val_accuracy improved from 0.96862 to 0.98745, saving model to /content/drive/MyDrive/Ali Amini/Fold1/TactileNet/Aug Saved Model/Config19.check


145/145 - 13s - loss: 0.0049 - accuracy: 0.9983 - val_loss: 0.0717 - val_accuracy: 0.9874 - lr: 5.0000e-04 - 13s/epoch - 90ms/step
Epoch 9/100

Epoch 9: val_accuracy did not improve from 0.98745
145/145 - 9s - loss: 0.0033 - accuracy: 0.9990 - val_loss: 0.0999 - val_accuracy: 0.9833 - lr: 5.0000e-04 - 9s/epoch - 65ms/step
Epoch 10/100

Epoch 10: val_accuracy did not improve from 0.98745
145/145 - 9s - loss: 0.0023 - accuracy: 0.9991 - val_loss: 0.2936 - val_accuracy: 0.9435 - lr: 5.0000e-04 - 9s/epoch - 65ms/step
Epoch 11/100

Epoch 11: val_accuracy did not improve from 0.98745
145/145 - 9s - loss: 0.0020 - accuracy: 0.9995 - val_loss: 0.0911 - val_accuracy: 0.9686 - lr: 5.0000e-04 - 9s/epoch - 65ms/step
Epoch 12/100

Epoch 12: val_accuracy did not improve from 0.98745
145/145 - 9s - loss: 6.0573e-04 - accuracy: 0.9999 - val_loss: 0.1925 - val_accuracy: 0.9017 - lr: 5.0000e-04 - 9s/epoch - 65ms/step
Epoch 13/100

Epoch 13: val_accuracy did not improve from 0.98745
145/145 - 9s - loss: 

145/145 - 13s - loss: 3.3765e-04 - accuracy: 0.9999 - val_loss: 0.0335 - val_accuracy: 0.9958 - lr: 5.0000e-04 - 13s/epoch - 87ms/step
Epoch 18/100

Epoch 18: val_accuracy did not improve from 0.99582
145/145 - 9s - loss: 7.5322e-04 - accuracy: 0.9997 - val_loss: 0.1142 - val_accuracy: 0.9686 - lr: 5.0000e-04 - 9s/epoch - 65ms/step
Epoch 19/100

Epoch 19: val_accuracy did not improve from 0.99582
145/145 - 9s - loss: 8.0122e-04 - accuracy: 0.9997 - val_loss: 0.0917 - val_accuracy: 0.9644 - lr: 5.0000e-04 - 9s/epoch - 65ms/step
Epoch 20/100

Epoch 20: val_accuracy did not improve from 0.99582
145/145 - 9s - loss: 0.0018 - accuracy: 0.9996 - val_loss: 0.0969 - val_accuracy: 0.9686 - lr: 5.0000e-04 - 9s/epoch - 65ms/step
Epoch 21/100

Epoch 21: val_accuracy did not improve from 0.99582
145/145 - 9s - loss: 0.0023 - accuracy: 0.9991 - val_loss: 0.0378 - val_accuracy: 0.9958 - lr: 5.0000e-04 - 9s/epoch - 65ms/step
Epoch 22/100

Epoch 22: val_accuracy did not improve from 0.99582
145/145 - 9

In [28]:
Result = np.array(Result)
np.save('/content/drive/MyDrive/Ali Amini/Fold1/TactileNet/Result.npy',Result)